In [ ]:
pip install selenium


SyntaxError: invalid syntax (1709309456.py, line 1)

In [12]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time


SyntaxError: invalid syntax (2440085156.py, line 9)

In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# 크롬 옵션 설정
options = Options()
# options.add_argument('--headless')  # 창 없이 실행할 때 사용
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 크롬 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 1. 페이지 접속
url = "https://grandpark.seoul.go.kr/animal/animalList/ko/S001002002003.do"
driver.get(url)
time.sleep(2)

# 2. '학문적 분류' 드롭다운에서 '조류강' 선택
select_element = driver.find_element(By.ID, "search_animal_cd")
select = Select(select_element)
select.select_by_value("0101030000")  # 조류강
time.sleep(1)

# 3. '검색' 버튼 클릭
search_button = driver.find_element(By.CSS_SELECTOR, "button.search-button")
search_button.click()
time.sleep(2)  # 조류 페이지 로딩 대기

print("✅ 조류강 페이지 진입 완료!")


✅ 조류강 페이지 진입 완료!


In [34]:
def parse_current_page(driver):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    animal_blocks = soup.select("div.gallery-detail")

    data = []

    for block in animal_blocks:
        name = block.select_one("a.gallery-title").text.strip()

        # 기본값 지정
        order = family = location = "정보 없음"

        li_tags = block.select("ul.circle-list.tbl > li")

        for li in li_tags:
            label = li.select_one("strong").text.strip()
            value = li.select_one("span").text.strip()

            if "목" in label:
                order = value
            elif "과" in label:
                family = value
            elif "위치" in label:
                location = value

        data.append({
            "이름": name,
            "목분류": order,
            "과분류": family,
            "공원내위치": location
        })

    return data


In [ ]:

all_data = []

for page in range(1, ): 
    # 페이지 수는 수동으로 입력
    url = f"https://grandpark.seoul.go.kr/animal/animalList/ko/S001002002003.do?pageIndex={page}&search_animal_cd=0101030000"
    driver.get(url)
    time.sleep(1)  # 로딩 대기
    page_data = parse_current_page(driver)
    all_data.extend(page_data)

# 브라우저 닫기
driver.quit()


In [36]:
df = pd.DataFrame(all_data)
df.to_csv("서울대공원_보유동물_파충류강정보.csv", index=False, encoding="utf-8-sig")

df.head()


,이름,목분류,과분류,공원내위치
0,팬케이크육지거북,거북목,땅거북과,동양관
1,솔방울도마뱀,뱀목,도마뱀과,동양관
2,유로메스틱스게리,뱀목,아가마과,동양관
3,시밀리스왕도마뱀,뱀목,왕도마뱀과,동양관
4,레이저백머스크터틀,거북목,-,곤충관
